# Deep Learning Preprocessing v2 

__Author:__ Jack Friedman <br>
__Date:__ 11/6/2023 <br>
__Purpose:__ Playground for converting data into position image

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Preprocessing_v2 import *
from DataLoader import load_data

In [ ]:
# Load data
[games_df, players_df, plays_df, tracking_df] = load_data()

In [ ]:
tracking_df_clean = preprocess_all_df(plays_df, games_df, players_df, tracking_df)

In [ ]:
# Check
for var in ['X_std', 'Y_std', 'Dir_std', 'O_std']:
    print(tracking_df_clean[var].describe())

In [ ]:
tracking_df_clean[(tracking_df_clean['X_std'] > 10) & (tracking_df_clean['X_std'] < 11)]['X_std'].unique()

In [ ]:
tracking_df_clean['event'].unique()

### frame cutoffs

In [ ]:
tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 80) & (tracking_df_clean['frameId'] <= tracking_df_clean[(tracking_df_clean['event'] == 'run') | (tracking_df_clean['event'] == 'handoff')]['frameId'].max())]

In [ ]:
# Filter for pre-handoff events 

# Step 1: Get the frames where handoff or run occurs 
frame_cutoffs = tracking_df_clean[(tracking_df_clean['event'] == 'run') | (tracking_df_clean['event'] == 'handoff')][['gameId', 'playId', 'frameId']].drop_duplicates()
frame_cutoffs = frame_cutoffs.rename(columns = {'frameId':'frame_cutoff'})
frame_cutoffs

In [ ]:
# Look at distribution fo cutoffs
frame_cutoffs['frame_cutoff'].describe()

In [ ]:
tracking_df_clean['frameId']

In [ ]:
# Step 2: Merge the frame cutoffs with the original dataframe and filter the frames before the cutoff
tracking_df_clean = pd.merge(tracking_df_clean, frame_cutoffs, on=['gameId', 'playId'])

In [ ]:
tracking_df_clean[tracking_df_clean['frameId'] <= tracking_df_clean['frame_cutoff']]['event'].unique()

### check passing plays

In [ ]:
possible_pass_plays = tracking_df_clean[tracking_df_clean['event'].str.contains('pass', na=False)][['gameId', 'playId']].drop_duplicates()

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.merge(plays_df, possible_pass_plays, on=['gameId', 'playId'], how='inner')

In [ ]:
len(tracking_df_clean[(tracking_df_clean['event'] == 'run') | (tracking_df_clean['event'] == 'pass_forward')][['gameId', 'playId']].drop_duplicates())

In [ ]:
plays_df[(plays_df['gameId'] == 2022090800) & (plays_df['playId'] == 80)]['playDescription']

### Check when 2 players are in the same matrix cell (same yard^2) in a play

In [38]:
# Keep track of number of conflicts when 2+ players are in a cell
num_conflicts = 0
pre_cutoff_rows = tracking_df_clean[tracking_df_clean['frameId'] <= tracking_df_clean['frame_cutoff']]
for (game_id, play_id), group_df in pre_cutoff_rows.groupby(['gameId', 'playId']):
    for frame_id, frame_df in group_df.groupby(['frameId']):
        # Create a blank matrix of the field
        image = np.zeros((120, 54))

        for _, row in frame_df.iterrows():
            if pd.isnull(row['nflId']):
                continue

            # Keep track of x and y locations of player (matrix indices)
            i = int(row['X_std'])
            j = int(row['Y_std'])

            image[i, j] += 1

            if image[i,j] > 1:
                num_conflicts += 1
                print("found a conflict in " + str((game_id, play_id, frame_id)) + " at " + str((i, j)))
                break

print('total conflicts: ' + str(num_conflicts))

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to

found a conflict in (2022090800, 101, 14) at (48, 23)
found a conflict in (2022090800, 101, 17) at (47, 25)
found a conflict in (2022090800, 146, 17) at (60, 26)
found a conflict in (2022090800, 146, 22) at (60, 25)
found a conflict in (2022090800, 191, 16) at (74, 28)
found a conflict in (2022090800, 191, 17) at (74, 26)
found a conflict in (2022090800, 191, 18) at (75, 29)
found a conflict in (2022090800, 299, 13) at (26, 25)
found a conflict in (2022090800, 299, 14) at (26, 25)
found a conflict in (2022090800, 299, 15) at (26, 22)
found a conflict in (2022090800, 299, 16) at (26, 22)
found a conflict in (2022090800, 299, 17) at (25, 28)
found a conflict in (2022090800, 299, 18) at (25, 28)
found a conflict in (2022090800, 299, 19) at (25, 28)
found a conflict in (2022090800, 343, 15) at (24, 33)
found a conflict in (2022090800, 343, 17) at (26, 28)
found a conflict in (2022090800, 393, 19) at (50, 24)


/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to

found a conflict in (2022090800, 896, 17) at (43, 20)
found a conflict in (2022090800, 933, 8) at (73, 33)
found a conflict in (2022090800, 933, 9) at (73, 33)
found a conflict in (2022090800, 933, 10) at (73, 33)
found a conflict in (2022090800, 933, 13) at (73, 32)
found a conflict in (2022090800, 933, 14) at (73, 32)
found a conflict in (2022090800, 933, 15) at (73, 32)
found a conflict in (2022090800, 933, 16) at (73, 33)
found a conflict in (2022090800, 933, 17) at (73, 33)
found a conflict in (2022090800, 933, 18) at (73, 34)
found a conflict in (2022090800, 933, 19) at (73, 34)
found a conflict in (2022090800, 933, 20) at (73, 34)
found a conflict in (2022090800, 1187, 13) at (49, 22)
found a conflict in (2022090800, 1187, 14) at (49, 22)
found a conflict in (2022090800, 1187, 15) at (49, 21)
found a conflict in (2022090800, 1254, 15) at (64, 28)
found a conflict in (2022090800, 1254, 16) at (64, 28)
found a conflict in (2022090800, 1254, 17) at (60, 29)
found a conflict in (202

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to

...

found a conflict in (2022092502, 1280, 4) at (55, 31)
found a conflict in (2022092502, 1301, 11) at (49, 30)
found a conflict in (2022092502, 1301, 12) at (49, 30)
found a conflict in (2022092502, 1301, 13) at (49, 30)
found a conflict in (2022092502, 1301, 14) at (49, 30)
found a conflict in (2022092502, 1301, 15) at (49, 30)
found a conflict in (2022092502, 1301, 16) at (45, 30)
found a conflict in (2022092502, 1301, 17) at (49, 24)
found a conflict in (2022092502, 1469, 12) at (39, 25)
found a conflict in (2022092502, 1567, 1) at (107, 30)
found a conflict in (2022092502, 1567, 2) at (107, 30)
found a conflict in (2022092502, 1567, 3) at (107, 30)
found a conflict in (2022092502, 1567, 5) at (107, 30)
found a conflict in (2022092502, 1567, 12) at (106, 22)
found a conflict in (2022092502, 1567, 13) at (108, 28)
found a conflict in (2022092502, 1567, 14) at (108, 28)
found a conflict in (2022092502, 1567, 15) at (108, 28)
found a conflict in (2022092502, 1567, 16) at (108, 22)
found 

/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to

found a conflict in (2022092502, 2149, 17) at (62, 28)
found a conflict in (2022092502, 2149, 18) at (63, 24)
found a conflict in (2022092502, 2149, 19) at (63, 24)
found a conflict in (2022092502, 2196, 16) at (73, 27)
found a conflict in (2022092502, 2196, 17) at (73, 27)
found a conflict in (2022092502, 2196, 18) at (73, 26)
found a conflict in (2022092502, 2196, 19) at (73, 26)
found a conflict in (2022092502, 2196, 20) at (73, 26)
found a conflict in (2022092502, 2314, 9) at (94, 29)
found a conflict in (2022092502, 2314, 10) at (94, 29)
found a conflict in (2022092502, 2314, 11) at (94, 29)
found a conflict in (2022092502, 2314, 12) at (94, 29)
found a conflict in (2022092502, 2314, 13) at (94, 32)
found a conflict in (2022092502, 2314, 14) at (95, 27)
found a conflict in (2022092502, 2314, 15) at (91, 29)
found a conflict in (2022092502, 2314, 16) at (94, 33)
found a conflict in (2022092502, 2337, 2) at (115, 5)
found a conflict in (2022092502, 2337, 3) at (115, 5)


/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for frame_id, frame_df in group_df.groupby(['frameId']):
/var/folders/nt/y2pysqbx2wg4rrlhdnq0djrc0000gn/T/ipykernel_65595/4203791356.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to

IndexError: index 120 is out of bounds for axis 0 with size 120

In [39]:
print(num_conflicts)

7595


In [ ]:
# found a conflict at (2022090800, 146, 15)
tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 146) & (tracking_df_clean['frameId'] == 15)][['X_std', 'Y_std']]

In [ ]:
int(28.963333)

In [ ]:
 # Create a blank matrix of the field
image = np.zeros((120, 54))

# found a conflict at (2022090800, 146, 15)
frame = tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 146) & (tracking_df_clean['frameId'] == 15)]
for _, row in frame.iterrows():
    if pd.isnull(row['nflId']):
        continue

    # Keep track of x and y locations of player (matrix indices)
    i = int(row['X_std'])
    j = int(row['Y_std'])

    image[i, j] += 1

    if image[i, j] > 1:
        print(i, j)

image

## Create Image of the field

Notes from soroush meeting

1. Channels - 0 to 1 - for num players
2. Ratio of teams- if all in same team (1, half-half is 0.5, 2/3 on one team) --> 0 to 1
3. Offense ratio- 0-1 ratio of offense (1 = only offense, 0 = nobody there)
4. Defense ratio- 0 -1 ratio of defense (1 = only defense, 0 = )
5. Velocity - average velocity and 
6. ariance of velocity in the cells 
6. Height - average height of players in the cell 0 - 1 (1 is tallest height)
7. Variance of the height 
8. Channel for whether the football is there


All channels should go to 0 to 1
- Every continuosu variable can be represented with 2 variables- average and variance
- 



List of Channels
1. Spatial - 0 to 1 for percentage of players on the field at this location (0 = nobody 1 = all 21 players at this spot) -- [will never reach 1 but need to scale for a ton of linemen being in one spot or a tackling dogpile with many players- can change upper bound]
2. Ball location: = 1 if ball at this location, = 0 if not
3. [DEPRECIATED] Team ratio - if all in same team (1, half-half is 0.5, 2/3 on one team) --> 0 to 1
4. Offense ratio - 0-1 ratio of offense (1 = only offense, 0 = nobody there, else = # offense / total people 
5. Defense ratio -  0 -1 ratio of defense (1 = only defense, 0 = nobody there, else = % of players in that cell on defense)
6. Velocity (average, standardized 0-1)
7. Velocity (variation, standardized 0-1)
8. Acceleration (average, standardized 0-1)
9. Acceleration (variation, standardized 0-1)
10. Net??? angle of player motion – standardization to not include 0?? 
11. Angle of player orientation??????
12. Height (average, standardized 0-1)
13. Height (variation, standardized 0-1)
14. Weight (average, standardized 0-1)
15. Weight (variation, standardized 0-1)


In [ ]:
single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 80) & (tracking_df_clean['frameId'] == 2)]
single_frame_data 

Notes on update formulae: <br>

Variables: <br>
$n_o$ = total # offensive players checked so far <br>
$n_d$ = total # defensive players checked so far <br>
$p_0$ = existing % at this spot <br>

note total players checked = $n_o + n_d$

1. Spatial layer (% of players on field at this location):
    - Formula to udpate = $\frac{(p_0)(n_o + n_d) + 1}{(n_o + n_d) + 1}$ =  $\frac{\text{(existing \% at this spot)} \text{(\# players checked)} + 1}{\text{(\# players checked)} + 1}$
3. Offese ratio:
    - Formula to update = $\frac{(p_0)(n_o) + 1}{n_o + 1}$ 

In [ ]:
pd.set_option('display.max_columns', None)
single_frame_data

In [ ]:
single_frame_data = tracking_df_clean[(tracking_df_clean['gameId'] == 2022090800) & (tracking_df_clean['playId'] == 80) & (tracking_df_clean['frameId'] == 2)]

# STEP 0: Create a blank matrix
image = np.zeros((120, 54, 15))

# STEP 1: Record ball location before looping through players in Channel 1
i = int(round(single_frame_data[single_frame_data['nflId'].isnull()]['X_std']))
j = int(round(single_frame_data[single_frame_data['nflId'].isnull()]['Y_std']))
image[i, j, 1] = 1

# STEP 2: Drop football from dataframe
single_frame_data = single_frame_data.dropna(subset=['nflId'])

# STEP 3: Populate player channels, person by person
num_o_players_checked = 0
num_d_players_checked = 0
for _, row in single_frame_data.iterrows():
    # Keep track of x and y locations of player (matrix indices)
    i = int(round(row['X_std']))
    j = int(round(row['Y_std']))
    total_num_players_checked = num_o_players_checked + num_d_players_checked

    # Channel 0: Record position of this player
    image[i, j, 0] = (image[i, j, 0] * total_num_players_checked + 1) / (total_num_players_checked + 1)

    # Channel 2: Record offensive player ratio
    num_o_players_checked += row['isOnOffense']  # update if we've checked an offensive player
    image[i, j, 0] = (image[i, j, 0] * num_o_players_checked) / (total_num_players_checked + 1)
    
    # Channel 3: Record defensive player ratio
    num_d_players_checked += (1 - row['isOnOffense'])  # update if we've checked a defensive player
    image[i, j, 0] = (image[i, j, 0] * num_d_players_checked) / (total_num_players_checked + 1)
    
    # Channel 4: Velocity (average, standardized 0-1)
    
    # 6. Velocity (average, standardized 0-1)
    # 7. Velocity (variation, standardized 0-1)
    # 8. Acceleration (average, standardized 0-1)
    # 9. Acceleration (variation, standardized 0-1)
    # 10. Net??? angle of player motion – standardization to not include 0?? 
    # 11. Angle of player orientation??????
    # 12. Height (average, standardized 0-1)
    # 13. Height (variation, standardized 0-1)
    # 14. Weight (average, standardized 0-1)
    # 15. Weight (variation, standardized 0-1)

In [ ]:
single_frame_data[['X_std', 'Y_std']]

In [ ]:
np.set_printoptions(threshold=np.inf)
image[:,:,0]